<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#一-处理train数据，只留用户id、电影id、评分" data-toc-modified-id="一-处理train数据，只留用户id、电影id、评分-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>一 处理train数据，只留用户id、电影id、评分</a></span></li><li><span><a href="#二-处理movie数据，电影id、年份、类型" data-toc-modified-id="二-处理movie数据，电影id、年份、类型-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>二 处理movie数据，电影id、年份、类型</a></span><ul class="toc-item"><li><span><a href="#1-提取年份" data-toc-modified-id="1-提取年份-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>1 提取年份</a></span></li><li><span><a href="#2-提取类别" data-toc-modified-id="2-提取类别-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>2 提取类别</a></span></li></ul></li></ul></div>

先做数据处理，我们准备使用的数据集形式如下：  
用户id、电影id、电影类别特征（独热化）、评分（标签）

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import math

plt.rcParams['font.family']=['Songti SC']
plt.rcParams['axes.unicode_minus'] = False

# pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.display.max_rows = 200

# np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
np.set_printoptions(threshold=np.inf)
np.set_printoptions(linewidth=100, suppress=True)

# 一 处理train数据，只留用户id、电影id、评分

In [2]:
train = pd.read_csv("../data/training_ratings.csv",usecols=[0,1,2],dtype={'userId':'category','movieId':'category','rating':'float'})
train.describe()

,rating
count,90743.000000
mean,3.501807
std,1.042895
min,0.500000
25%,3.000000
50%,3.500000
75%,4.000000
max,5.000000


In [3]:
train.describe(include='category')

,userId,movieId
count,90743,90743
unique,610,9362
top,414,356
freq,2428,299


In [4]:
train

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
90738,610,166534,4.0
90739,610,168248,5.0
90740,610,168250,5.0
90741,610,168252,5.0


# 二 处理movie数据，电影id、年份、类型

In [5]:
movie = pd.read_csv("../data/movies.csv")
movie

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


## 1 提取年份

发现年份数据有缺失，且我们觉得年份可能不是特别重要，所以不加入年份特征

In [17]:
# movie['year'] = movie['title'].apply(lambda x: x.strip()[-5:-1])

In [18]:
# movie['year']

## 2 提取类别

In [6]:
genres=set()
for m in movie.genres:
    genres.update(g for g in m.split('|')) 
genres = sorted(genres)

In [7]:
genres

['(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [8]:
len(genres)

20

In [9]:
for genre in genres:
    movie[genre] = [genre in genres2.split('|') for genres2 in movie.genres]
    movie[genre] = movie[genre].astype(int)

In [10]:
new_movie = movie.drop(columns=['genres','(no genres listed)','title'])

In [17]:
new_movie

,movieId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9738,193583,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9739,193585,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9740,193587,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3, random_state = 1)
new_movie['label'] = km.fit_predict(new_movie[new_movie.columns[1:]])

In [21]:
new_movie.to_csv('../result/new_movie.csv',index=None,float_format = '%.1f')

In [20]:
new_movie['label'].value_counts()

0    3699
1    3358
2    2685
Name: label, dtype: int64

In [41]:
train.movieId = train.movieId.astype('int')

In [42]:
df = pd.merge(train,new_movie,on='movieId')

In [43]:
df

,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,1,4.0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,1,4.5,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17,1,4.5,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,18,1,3.5,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90738,610,152372,3.5,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
90739,610,158721,3.5,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
90740,610,160341,2.5,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
90741,610,160836,3.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [46]:
df.to_csv('../result/train.csv',index=None,float_format = '%.1f')

统计信息
- 这个电影的平均分、个数
- 这个用户打出的平均分、个数

In [49]:
df.groupby(['userId'])['rating'].mean()

userId
1      4.373206
10     3.265873
100    3.939850
101    3.509091
102    3.440000
         ...   
95     4.076159
96     3.957143
97     4.218750
98     3.975904
99     3.875000
Name: rating, Length: 610, dtype: float64